# Misspelling detection and correction

**NOTE**: This notebook depends upon the the Retrotech dataset. If you have any issues, please rerun the [Setting up the Retrotech Dataset](../ch4/1.ch4-setting-up-the-retrotech-dataset.ipynb) notebook.

In [ ]:
import pandas as pd
import nltk
from collections import defaultdict
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
import numpy as np
import re
nltk.download('stopwords')

In [ ]:
signals_data= pd.read_csv("../data/retrotech/signals.csv")


In [ ]:
### get keyword only 
is_query =  signals_data['type']=='query'
signals_data  = signals_data[is_query]

### Step 1: Tokenize queries and count word frequencies. 
Check word frequency distribut quantiles. The quantile will help decide cut off point for potential misspells and corrections. 

In [ ]:
stop_words = set(stopwords.words('english'))
word_list = defaultdict(int)

#for query in signal_sample["query_s"]:
for query in signals_data["target"]:
    tokenizer = RegexpTokenizer(r'\w+') 
    tokened   = tokenizer.tokenize(str(query.lower()))
    
    for token in tokened:
        if token not in stop_words and len(token) > 3 and not token.isdigit():  #drop stopwords and digit only tokens
            # and only consider token length > 3, since hard to judge whether a very short token is misspelled or not
            word_list[token] += 1

In [11]:
np.quantile(np.array(list(word_list.values())), [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])

array([  5. ,   7. ,   8. ,  12. ,  16. ,  25. ,  47. , 143. , 335.1])

### Step 2: compute metadata needed for word matching. 
consider word with low count as misspelling condidates, with high count as correctly spelled candidates. 

In [23]:
misspell_candidates = []
correction_candidates = []
misspell_counts = []
correction_counts = []
misspell_length = []
correction_length = []
misspell_initial = []
correction_initial = []

for k, v in word_list.items():
    if v <=5 : #if v == 1:  # this number based on quantile analysis and the data set, more-likely with user-behvaiour data set to be 1
        misspell_candidates.append(k)
        misspell_counts.append(v)
        misspell_length.append(len(k))
        misspell_initial.append(k[0])
    if v >= 9:
        correction_candidates.append(k)
        correction_counts.append(v)
        correction_length.append(len(k))
        correction_initial.append(k[0])

In [24]:
misspell_candidates_df = pd.DataFrame({"misspell":misspell_candidates, "misspell_counts":misspell_counts, "misspell_length":misspell_length,"initial":misspell_initial})
correction_candidates_df = pd.DataFrame({"correction":correction_candidates, "correction_counts":correction_counts, "correction_length":correction_length,"initial":correction_initial})

In [26]:
misspell_candidates_df.head(30)

,misspell,misspell_counts,misspell_length,initial
0,matthew,5,7,m
1,mystery,5,7,m
2,scooters,5,8,s
3,wonders,5,7,w
4,behind,5,6,b
5,marvin,5,6,m
6,gaye,5,4,g
7,shallow,5,7,s
8,meddle,5,6,m
9,threes,5,6,t


### Step 3: Find potential matches 
based on edit distance and whether word initial is the same or not. 

In [27]:
def good_match(len1, len2, edit_dist): #allow longer words have more edit distance
    match = 0
    min_length = min(len1, len2)
    if min_length < 8:
        if edit_dist == 1: match = 1
    elif min_length < 11:
        if edit_dist <= 2: match = 1
    else:
        if edit_dist == 3: match = 1
    return match

In [28]:
matches_candidates = pd.merge(misspell_candidates_df, correction_candidates_df, on="initial")
#join missepll list with correction list based on whether they share the same initials to reduce matching time. 
matches_candidates["edit_dist"] = matches_candidates.apply(lambda row: nltk.edit_distance(row.misspell,row.correction), axis=1)
matches_candidates["good_match"] = matches_candidates.apply(lambda row: good_match(row.misspell_length, row.correction_length, row.edit_dist),axis=1)

In [29]:
matches = matches_candidates[matches_candidates["good_match"] == 1].drop(["initial","good_match"],axis=1)

### Step 4: rank potential matched corrections 
based on edit distance and correction word frequency. shorter edit distance and higher word count will be prefered. only the top one correction is selected for final matching. 

In [30]:
matches.sort_values(by=['misspell', 'edit_dist', 'correction_counts'], ascending=[True, True, False])
matches_final = matches.groupby('misspell').first().reset_index()

In [31]:
matches_final.head(10)

,misspell,misspell_counts,misspell_length,correction,correction_counts,correction_length,edit_dist
0,1490lmt,5,7,1450lmt,32,7,1
1,500d,5,4,550d,21,4,1
2,acessories,5,10,accessories,1272,11,1
3,adam,5,4,adams,33,5,1
4,aladin,5,6,aladdin,24,7,1
5,alianware,5,9,alienware,429,9,1
6,alienwear,5,9,alienware,429,9,2
7,backups,5,7,backup,428,6,1
8,bbr6,5,4,bbr9,12,4,1
9,belly,5,5,bello,19,5,1
